In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import os
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ['0', '5', '10', '19', '28']
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

def load_image(image_name):
    image = Image.open(image_name)
    image = image.convert("RGB")
    image = loader(image).unsqueeze(0)
    return image.to(device)

def convert_to_jpg(image_path):
    img = Image.open(image_path)
    file_name, file_extension = os.path.splitext(image_path)

    if file_extension.lower() != ".jpg":
        new_image_path = f"{file_name}.jpg"
        img = img.convert("RGB") 
        img.save(new_image_path, "JPEG", quality=90)
        return new_image_path
    else:
        return image_path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 356

# Add normalization to the loader
loader = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)



model = VGG().to(device).eval()

original_img_name = "image1.jpeg"
style_img_name = "style.jpg"

original_image_name = convert_to_jpg(original_img_name)
style_img_name = convert_to_jpg(style_img_name)

original_img = load_image(original_img_name)
style_img = load_image(style_img_name)
generated = original_img.clone().requires_grad_(True)

# Hyperparameters
total_steps = 3000
learning_rate = 0.008
alpha = 10
beta = 0.1
optimizer = optim.Adam([generated], lr=learning_rate)

for step in range(total_steps):
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(
            generated_features, original_img_features, style_features
    ):
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

        # Compute Gram matrix
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated1.jpg")
        

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor(13980285., device='cuda:0', grad_fn=<AddBackward0>)
tensor(65388.3242, device='cuda:0', grad_fn=<AddBackward0>)
tensor(33250.1211, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24922.9258, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20617.1309, device='cuda:0', grad_fn=<AddBackward0>)
tensor(18044.7949, device='cuda:0', grad_fn=<AddBackward0>)
tensor(16366.6475, device='cuda:0', grad_fn=<AddBackward0>)
tensor(15273.4023, device='cuda:0', grad_fn=<AddBackward0>)
tensor(14567.6299, device='cuda:0', grad_fn=<AddBackward0>)
tensor(15871.2949, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12596.1797, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13205.7549, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13466.5195, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11967.5537, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11119.2012, device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
models.vgg19(pretrained=True).features

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [ ]:
!pip install SSIM-PIL

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image
from SSIM_PIL import compare_ssim

image1 = Image.open('/content/image.jpeg')
target_size = (356, 356)
# Resize the image
resized_img = image1.resize(target_size)
image2 = Image.open('/content/generated1.jpg')

#  Compare images using CPU-only version
value = compare_ssim(resized_img, image2, GPU=False)
print(value)

0.3361212368225351


In [ ]:
np.mean(np.abs(np.array(resized_img) - np.array(image2)))/255

0.47492231290529935